# Проект для «Викишоп»

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Первичный-анализ-данных" data-toc-modified-id="Первичный-анализ-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Первичный анализ данных</a></span></li><li><span><a href="#Очистка-текста" data-toc-modified-id="Очистка-текста-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Очистка текста</a></span></li><li><span><a href="#Лемматизация-текста" data-toc-modified-id="Лемматизация-текста-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Лемматизация текста</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Оценка-важности-слова-(TF-IDF)" data-toc-modified-id="Оценка-важности-слова-(TF-IDF)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Оценка важности слова (TF-IDF)</a></span></li><li><span><a href="#Деление-данных-на-выборки" data-toc-modified-id="Деление-данных-на-выборки-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Деление данных на выборки</a></span></li><li><span><a href="#TF-IDF" data-toc-modified-id="TF-IDF-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>TF-IDF</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Модель-Random-Forest" data-toc-modified-id="Модель-Random-Forest-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Модель Random Forest</a></span></li><li><span><a href="#Модель-Decision-Tree" data-toc-modified-id="Модель-Decision-Tree-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Модель Decision Tree</a></span></li><li><span><a href="#Сравнение-моделей" data-toc-modified-id="Сравнение-моделей-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Сравнение моделей</a></span></li></ul></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Импортируем все необходимые библиотеки

In [1]:
import numpy as np

import pandas as pd

import re

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as nltk_stopwords

import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

RANDOM = 12345

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...

[nltk_data]   Package stopwords is already up-to-date!
































⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the

full pipeline package name 'en_core_web_sm' instead.



     |████████████████████████████████| 13.9 MB 7.5 MB/s eta 0:00:01































✔ Download and installation successful

You can now load the package via spacy.load('en_core_web_sm')


In [2]:
from tqdm import tqdm
import spacy

Все необходимые для дальнейшей работы библиотеки импортированы. Приступим к рассмотрению данных.

### Первичный анализ данных

Откроем наш датасет и сохраним его в переменной df. Затем выведем на экран первые 5 строк датасета, а также информацию о нем с помощью метода info(). А также сразу проверим датасет на предмет пропусков.

In [3]:
df = pd.read_csv('/datasets/toxic_comments.csv')

In [4]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 159292 entries, 0 to 159291

Data columns (total 3 columns):

 #   Column      Non-Null Count   Dtype 

---  ------      --------------   ----- 

 0   Unnamed: 0  159292 non-null  int64 

 1   text        159292 non-null  object

 2   toxic       159292 non-null  int64 

dtypes: int64(2), object(1)

memory usage: 3.6+ MB


In [6]:
df.isna().sum()

Unnamed: 0    0
text          0
toxic         0
dtype: int64

Как мы видим из полученного - наш датасет состоит из 3-х колонок (номер, текст комментария и оценка токсичности (0 или 1). Следовательно, перед нами задача классификации - нам необходимо обучить модель, которая будет определять токсичный комментарий или нет (0 или 1). Также в датасете отсутствуют пропуски. Выведем на экран колонку "Текст", чтобы посмотреть в каком состоянии находятся в нем данные.

In [7]:
df.text

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159287    ":::::And for the second time of asking, when ...
159288    You should be ashamed of yourself \n\nThat is ...
159289    Spitzer \n\nUmm, theres no actual article for ...
159290    And it looks like it was actually you who put ...
159291    "\nAnd ... I really don't think you understand...
Name: text, Length: 159292, dtype: object

Мы видим, что текст на английском языке, имеются различные разделители, не ровные отступы, рандомные символы. Ото всего этого необходимо избавиться перед тем, как дальше работать с текстом

### Очистка текста

От лишних символов текст очистят регулярные выражения. Это инструмент для поиска слова или числа по шаблону. Он определяет, из каких частей состоит строка и какие в них символы. Воспользуемся функцией re.sub(), в которую передадим символы английского алфавита (заглавные и строчные), которые нужно заменить на пробелы. Затем в такую же функцию передадим разделители, которые встречаются в наших комментариях, и также заменим их на пробелы. Методом join() объединим получившиеся элементы в строки. 

In [8]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    text = re.sub(r'(?:\n|\r)', ' ', text)
    text = text.lower()
    return " ".join(text.split())

Теперь заменим столбец "Текст" на столбец "Текст" очищенный и посмотрим что получилось.

In [9]:
df['text'] = df['text'].apply(clear_text)

In [10]:
df.text

0         explanation why the edits made under my userna...
1         d aww he matches this background colour i m se...
2         hey man i m really not trying to edit war it s...
3         more i can t make any real suggestions on impr...
4         you sir are my hero any chance you remember wh...
                                ...                        
159287    and for the second time of asking when your vi...
159288    you should be ashamed of yourself that is a ho...
159289    spitzer umm theres no actual article for prost...
159290    and it looks like it was actually you who put ...
159291    and i really don t think you understand i came...
Name: text, Length: 159292, dtype: object

Как мы видим - лишние символы, отступы и пробелы убрались. Текст выглядит пригодным для дальнейшей работы с ним. Перейдем к лемматизации.

### Лемматизация текста

Воспользуемся функцией spaCy для проведения лемматизации текста, т.е. преобразования слов в леммы (исходные слова).
Заранее мы загрузили библиотеку spaCy, а также словарь текста на английском - т.к. у нас комментарии на английском языке. 

Создадим переменную corpus, в которую передадим значения столбца "текст" нашего датасета, а затем напишем функцию лемматизации, которая возвращает слову исходную форму. Затем пройдемся списком по переменной corpus, чтобы провести полную лемматизацию.

В конце заменим исходный столбец "текст" лемматизированным.

In [11]:
corpus = df['text'].values

In [12]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
def lemmatize(text):
    doc = nlp(text)
    lemm_text = " ".join([token.lemma_ for token in doc])
    return lemm_text

lemm=[]
for i in tqdm(range(len(corpus))):
    lemm.append(lemmatize(corpus[i]))

    
df['text'] = pd.Series(lemm, index=df.index)

100%|██████████| 159292/159292 [25:45<00:00, 103.05it/s]


Посмотрим что получилось.

In [14]:
df

,Unnamed: 0,text,toxic
0,0,explanation why the edit make under my usernam...,0
1,1,d aww he match this background colour I m seem...,0
2,2,hey man I m really not try to edit war it s ju...,0
3,3,more I can t make any real suggestion on impro...,0
4,4,you sir be my hero any chance you remember wha...,0
...,...,...,...
159287,159446,and for the second time of ask when your view ...,0
159288,159447,you should be ashamed of yourself that be a ho...,0
159289,159448,spitzer umm there s no actual article for pros...,0
159290,159449,and it look like it be actually you who put on...,0


Текст лемматизирован.

Таким образом, мы проанализировали исходные данные - оказалось, что они чистые. Затем мы перешли к обработке самих текстов комментариев - для начала мы избавились от ненужых пробелов, символов-разделителей, случайных символов. А затем провели лемматизацию текстов, т.е. привели слова к изначальной форме. 

Как и говорилось ранее - данная задача классификации, а следовательно, будем использовать для ее решения модели классификации. Приступим.

## Обучение

### Оценка важности слова (TF-IDF)

Непосредственно перед подготовкой к обучению моделей и самим обучением, выполним еще один важный шаг - сделаем оценку важности слова (TF-IDF векторизацию). TF отвечает за количество упоминаний слова в отдельном тексте, а IDF отражает частоту его употребления во всём корпусе. Для начала загрузим словарь "стоп-слов" на английском языке из библиотеки nltk.

In [19]:
stopwords = set(nltk_stopwords.words('english'))

Теперь данные необходимо разделить на выборки.

### Деление данных на выборки

В нашей задачи целевой признак - это колонка "токсичность", а сопутствующие - все остальные колонки (в частности у нас это только колонка "текст"). Разделим данные на 2 выборки - тренировочную и тестовую в пропорции 75/25, а при обучении моделей будем использовать кросс-валидацию. После выведем на экран размеры выборок.

In [20]:
X = df['text']
y = df['toxic'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = RANDOM)

print(X_train.shape, y_train.shape) 
print(X_test.shape, y_test.shape)

(119469,) (119469,)

(39823,) (39823,)


Данные разделены на выборки. Продолжим заниматься TF-IDF векторизацией.

### TF-IDF

Мы в первую очередь разделили данные, так как если данные уже разделены на обучающую и тестовую выборки, функцию fit_tramsform() необходимо запускать только на обучающей выборке. На тестовой мы запустим только функцию transform(). Иначе тестирование будет нечестным: в модели будут учтены частоты слов из тестовой выборки.

Создадим счетчик, указав в нем стоп слова.

In [21]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 

Теперь воспользуемся, как говорилось выше, функцией fit_transform() на тренировочной выборке и функцией transform() на тестовой. Сохраним полученные результаты в новые переменные.

In [22]:
tf_idf_train = count_tf_idf.fit_transform(X_train)
tf_idf_test = count_tf_idf.transform(X_test)

Подготовка к обучению моделей завершена.

Для решения поставленной задачи воспользуемся 3-мя моделями:
- Логистическая регрессия
- Случайный лес
- Дерево решений

Лучшую модель будем определять по метрике 'f1', которая по условию задачи не должна быть меньше 0,75. Приступим.

### Логистическая регрессия

In [26]:
pipeline = Pipeline([(('model', LogisticRegression(random_state=RANDOM, solver='liblinear', max_iter=300)))])

param_grid = [{'model__penalty' : ['l1', 'l2']}]
glr = GridSearchCV(pipeline, param_grid=param_grid, scoring='f1', cv=5, verbose=True, n_jobs=-1)
glr.fit(tf_idf_train, y_train)
best_params_lr = glr.best_params_
best_score_lr = glr.best_score_
print("Лучшие гиперпараметры для модели:")
print()
print(best_params_lr)
print()
print("Лучшая метрика F1 с лучшими гиперпараметрами:")
print()
print(best_score_lr)

Fitting 5 folds for each of 2 candidates, totalling 10 fits

Лучшие гиперпараметры для модели:



{'model__penalty': 'l1'}



Лучшая метрика F1 с лучшими гиперпараметрами:



0.7649342813372237


### Модель Random Forest

In [27]:
model = RandomForestClassifier(random_state=RANDOM, n_jobs=-1) 
est = [x for x in range(10, 200, 50)]
params_rf = [{'n_estimators': est,
              'max_depth':[2, 10]}]

grf = GridSearchCV(model, params_rf, scoring='f1', cv=5, verbose=True)
grf.fit(tf_idf_train, y_train)
best_params_rf = grf.best_params_
best_score_rf = grf.best_score_
print("Лучшие гиперпараметры для модели:")
print()
print(best_params_rf)
print()
print("Лучшая метрика F1 с лучшими гиперпараметрами:")
print()
print(best_score_rf)


Fitting 5 folds for each of 8 candidates, totalling 40 fits

Лучшие гиперпараметры для модели:



{'max_depth': 10, 'n_estimators': 10}



Лучшая метрика F1 с лучшими гиперпараметрами:



0.0004939626969726234


### Модель Decision Tree

In [28]:
model = DecisionTreeClassifier(random_state=RANDOM) 
depth = [None] + [i for i in range(2, 10)]
params = [{'max_depth':depth}]

gdt = GridSearchCV(model, params, scoring='f1', cv=5, verbose=True,  n_jobs=-1)
gdt.fit(tf_idf_train, y_train)
best_params_dt = gdt.best_params_
best_score_dt = gdt.best_score_
print("Лучшие гиперпараметры для модели:")
print()
print(best_params_dt)
print()
print("Лучшая метрика F1 с лучшими гиперпараметрами:")
print()
print(best_score_dt)

Fitting 5 folds for each of 9 candidates, totalling 45 fits

Лучшие гиперпараметры для модели:



{'max_depth': None}



Лучшая метрика F1 с лучшими гиперпараметрами:



0.7126063136009069


### Сравнение моделей

Сравним получившиеся результаты в таблице.

In [29]:
index = ['LogisticRegression',
         'RandomForestClassifier',
         'DecisionTreeClassifier']
scores = {'Метрика F1 на тренировочной выборке с кросс-валидацией':[best_score_lr,
                                                                    best_score_rf,
                                                                    best_score_dt]}

res = pd.DataFrame(data=scores, index=index)
res

,Метрика F1 на тренировочной выборке с кросс-валидацией
LogisticRegression,0.764934
RandomForestClassifier,0.000494
DecisionTreeClassifier,0.712606


Наибольшая метрика F1 оказалась у модели Логистической Регресии - 0,76. Ее мы и проверим на тестовой выборке.

## Тестирование

Протестируем нашу модель с наилучшей метрикой F1.

In [30]:
model = LogisticRegression(random_state=RANDOM, penalty = 'l1', solver='liblinear', max_iter=300)
model.fit(tf_idf_train, y_train)
preds = model.predict(tf_idf_test)
print('Метрика F1 на тестовой выборке составила:', f1_score(y_test, preds))

Метрика F1 на тестовой выборке составила: 0.7894883980989656


Проверим модель на адекватность - для этого сравним ее с константной (dummy) моделью.

In [31]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(tf_idf_train, y_train)
dummy_pred = dummy_clf.predict(tf_idf_test)
dummy_f1 = f1_score(y_test, dummy_pred)
print('Метрика F1 для константной модели на тестовой составляет:', 
      dummy_f1)

Метрика F1 для константной модели на тестовой составляет: 0.0


Из этого можно сделать, что наша модель адекватна.

## Выводы

Перед нами стояла задача по обучению модели классифицировать комментарии интернет-магазина "Викишоп" на позитивные и негативные. Так как у магазина появился новый сервис - пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах, т.е. клиенты предлагают свои правки и комментируют изменения других. Магазину нужен был инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. Для этого в нашем распоряжении был набор данных с разметкой о токсичности правок. ТО есть нам надо получить на выходе токсичный комментарий или нет (0 или 1), а следовательно, перед нами стояла задача классификации.

Также стоило учесть, что метрика качества F1 должна быть не меньше 0.75.

Мы выгрузили интересующий нас датасет и провели его предварительную обработку. Мы проанализировали исходные данные - оказалось, что они чистые. Затем мы перешли к обработке самих текстов комментариев - для начала мы избавились от ненужых пробелов, символов-разделителей, случайных символов. А затем провели лемматизацию текстов, т.е. привели слова к изначальной форме.

Затем, для дальнейшего оубчения нами было выбрано 3 модели классификации: 
- Логистическая регрессия
- Случайный лес
- Дерево решений

Перед подготовкой к обучению моделей и самим обучением, выполним еще один важный шаг - сделаем оценку важности слова (TF-IDF векторизацию). Затем мы разделили данные на 2 выборки - тренировочную и тестовую в пропорции 75/25, а при обучении моделей использовали кросс-валидацию.

После разделения данных мы использовали функцию fit_tramsform() только на обучающей выборке. На тестовой мы запустили только функцию transform() - иначе тестирование было бы нечестным: в модели были бы учтены частоты слов из тестовой выборки.

Затем мы выделили целевой и сопутствующие признаки: целевой признак - это колонка "токсичность", а сопутствующие - все остальные колонки (в частности у нас это только колонка "текст").

Далее мы приступили непосредственно к обучению моделей на тренировочной выборке. Для поиска лучших гиперпараметров была использована функция GridSearchCV(). Итого, после обучения мы получили следующие метрики качества F1:
- Логистическая регрессия 0,764
- Случайный лес 0,0004
- Дерево решений 0,712

Как лучшая модель, была выбрана модель Логистической регрессии. Ее мы и протестировали.

На тестовой выборке метрика качества F1 равна 0,789, что входит в допустимый предел. Также мы проверили модель на адекватность с помощью константной модели - наша модель оказалась адекватна.

Таким образом, для решения стоящей перед заказчиком задачи, лучше всего подходит модель Логистической регрессии с подобранными нами гиперпараметрами.